In [1]:
import os
import pandas as pd
import numpy as np
import openai

from dotenv import load_dotenv

In [2]:
load_dotenv()

openai.organization = os.environ.get('OPENAI_ORG')
openai.api_key = os.environ.get('OPENAI_KEY')

In [4]:
# load the clean source data

folder = '../data'

df = pd.read_csv(f'{folder}/hanson_df.csv')
df = df[df['author'] == 'Robin Hanson'].copy()

df.head()

,date,author,title,body,url
1,2006-11-20,Robin Hanson,How To Join,How To Join How can we better believe what i...,https://www.overcomingbias.com/p/introductionhtml
2,2006-11-20,Robin Hanson,Hide Sociobiology Like Sex?,Hide Sociobiology Like Sex? Self-interest go...,https://www.overcomingbias.com/p/does_sociobio...
3,2006-11-21,Robin Hanson,The Movie “Click”,The Movie “Click” One way to discourage bias...,https://www.overcomingbias.com/p/click_christm...
5,2006-11-22,Robin Hanson,Beware Heritable Beliefs,Beware Heritable Beliefs Some of the differe...,https://www.overcomingbias.com/p/beware_herita...
6,2006-11-23,Robin Hanson,A 1990 Corporate Prediction Market,A 1990 Corporate Prediction Market Betting m...,https://www.overcomingbias.com/p/first_known_b...


In [40]:
# we'll try a sample size of 300 (203 due to chatgpt overload error) to fine-tine
# the model
question_samples = df['body'].sample(300).values

indices = [i for i, n in enumerate(question_samples) if len(n) >= 10000]
questions_filtered = [question_samples[i] for i in range(len(question_samples)) if i not in indices]

In [ ]:
# using chatgpt to generate sample questions to serve as the prompt 
# for fine-tuning
questions = []

for body in questions_filtered:
    
    prompt = f'SAMPLE TEXT : {body}. Please provide a question that would elicit the foregoing answer. One sentence.'
    
    messages = messages = [        
        {"role": "system", "content": "You are the best question-asker in the world."},        
        {"role": "user", "content": f"{prompt}"}    ]
    
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      temperature = 0.8,
      max_tokens = 2000,
      messages = messages
    )
    
    content_ = completion.choices[0].message.content
    
    questions.append(content_)

In [50]:
# creating the fine-tuning dataframe with columns for prompt and response
fine_tuning_df = (
    pd.DataFrame(
        {'prompt' : questions[:203], 
         'completion' : questions_filtered[:203]})
)

fine_tuning_df.head()

,prompt,completion
0,What percentage of the 51 scholars/pundits sur...,Policy vs. Meta-Policy What is our main prob...
1,What is the key tech premise in Altered Carbon...,The Ems of Altered Carbon People keep sugges...
2,What is the author's perspective on the percep...,"Social Scientists Know Lots For me, the year..."
3,What is the role of strong and clear evidence ...,When Does Evidence Win? Consider a random ar...
4,What is the role of standardization in the dev...,Shared AI Wins Almost every new technology c...


In [52]:
# saving the fine tuning dataset to a json file
fine_tuning_df.to_json('fine_tuning_test.jsonl')

I preprocessed and trained the model in the command line using the following commands:  

1. openai tools fine_tunes.prepare_data -f fine_tuning_test.jsonl  
2. set OPENAI_API_KEY=MY_API_KEY  
3. openai api fine_tunes.create -t "fine_tuning_test_prepared.jsonl" -m "curie"  
  
I used the OpenAI documentation: https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset  
And a blog here: https://norahsakal.com/blog/fine-tune-gpt3-model

In [24]:
# testing the new model's outputs
model_id = "curie:ft-independent-projects-2023-05-05-18-00-14"

# prompt
prompt = "Should people ever agree to disagree?"

# instance of Completion with the model_id of our custom model
completion = openai.Completion.create(
    model=model_id,
    prompt=prompt,
    temperature=0.5, 
    max_tokens=2000)

# printing the test
print(completion.choices[0].text)

 Should People Ever Agree To Disagree?  Some people think it’s a good idea for people to agree on some things, but disagree on others. For example, they agree that democracy is better than monarchy, but disagree on what the best democratic system is. I agree that it is a good idea for people to have some such divisions of opinion, so that they can discuss the issues in depth. But I don’t think people should ever agree to disagree on all issues.I’m not saying that people should all agree on the same things. Different people should probably have different opinions on this subject. But I’m saying that people should all agree on the same thing: namely, that they should never ever agree to disagree. People should always feel free to disagree with anyone. When people disagree, they should feel free to fight and even kill each other. That is the only way to make sure that no one ever gets too convinced that they are right.
“People should never agree to disagree”?  I’m not sure I’ve ever said 

In [ ]:
# welp, at least i fine-tuned a model and it only took a few minutes. i'll try again with 
# shorter 'completions' in the training set. this one is totally off the rails because the
# completions are much too long and insufficiently constrained by the prompts. 